---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    import operator
    graph_type = []
    
    for k,graph in enumerate(P1_Graphs):
        
        # Meausures to determine type of network model
        avg_cc = nx.average_clustering(graph)
        avg_short_path = nx.average_shortest_path_length(graph)
        max_degrees = sorted(graph.degree().items(),key = operator.itemgetter(1),reverse = True)[0:5]
        max_degrees = [m[-1] for m in max_degrees]
        
        # Network classification
        if (max_degrees[0] > 50):
            graph_type.append('PA')
        else:
            if (avg_cc > 0.1):
                graph_type.append('SW_L')
            else:
                graph_type.append('SW_H')
        
#         print('Graph ' + str(k+1) + ':' )
#         print(' + Avg CC: ', avg_cc)
#         print(' + Avg Short Path: ',avg_short_path)
#         print(' + Max degrees: ', max_degrees)
#         print()
    
    return graph_type


In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [6]:
# G_net = G.nodes(data = True)
# G_df = pd.DataFrame(index = G.nodes())
# G_df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
# G_df['degree'] = pd.Series(G.degree())
# G_df['CC'] = pd.Series(nx.clustering(G))
# G_df['Closeness'] = pd.Series(nx.closeness_centrality(G))
# G_df['BtwCentrality'] = pd.Series(nx.betweenness_centrality(G))
# G_df['ManageSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
# G_df.head(25)

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
def salary_predictions():
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler,RobustScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve,auc
    ##############################################################################
    # DATA LOADING
    #############################################################################
    
    G_net = G.nodes(data = True)
    G_df = pd.DataFrame(index = G.nodes())
    G_df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    G_df['degree'] = pd.Series(G.degree())
    G_df['CC'] = pd.Series(nx.clustering(G))
    G_df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    G_df['BtwCentrality'] = pd.Series(nx.betweenness_centrality(G))
    G_df['ManageSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    
    ##############################################################################
    # PRE - PROCESSING
    ##############################################################################
    
    # Drop NaN values
    G_df_cleaned = G_df.dropna()
    
    ##############################################################################
    # MODEL BUILDING --> 'SVC' Model Chosen
    ##############################################################################
    
    # Step 1: Defining and scaling X and y #
    
    ## Training
    X = pd.DataFrame( G_df_cleaned[['Department','degree','CC','Closeness','BtwCentrality']])
    y = pd.DataFrame( G_df_cleaned['ManageSalary'])
    
    ## Validations
    X_val = pd.DataFrame(G_df[G_df.isnull().any(axis=1)][['Department','degree','CC','Closeness','BtwCentrality']])
    y_val = pd.DataFrame(G_df[G_df.isnull().any(axis=1)]['ManageSalary'])
    
    # Aditional Step: Normalizing and scaling
    scaler = RobustScaler()
    
    X_scaled = scaler.fit_transform(X)
    X_val_scaled = scaler.fit_transform(X_val)
    
    # Step 2: Train&Test Split #
    
    X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,random_state = 42)
    
    # Step 3: Model definition #
    # clf = LogisticRegression()
    clf = RandomForestClassifier(n_estimators = 25)
    
    # Step 4: Model fitting #
    clf.fit(X_scaled,y)
    
    # Step 5: Model prediction and evaluation #
    
    ## Test
    y_pred = clf.predict_proba(X_test)[:,-1]

    fpr,tpr, thresh = roc_curve(y_test,y_pred)
    auc_score = auc(fpr,tpr)
    
#     # Plotting
#     import matplotlib.pyplot as plt
#     %matplotlib notebook
#     plt.figure()
#     plt.xlim([-0.01, 1.00])
#     plt.ylim([-0.01, 1.01])
#     plt.plot(fpr, tpr, lw=3, label='ROC curve (area = {:0.2f})'.format(auc_score))
#     plt.xlabel('False Positive Rate', fontsize=16)
#     plt.ylabel('True Positive Rate', fontsize=16)
#     plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
#     plt.legend(loc='lower right', fontsize=13)
#     plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
#     plt.axes().set_aspect('equal')
#     plt.show()
 
    y_pred_val = pd.DataFrame( clf.predict_proba(X_val_scaled), index = X_val.index)
    
    return y_pred_val[1] #, auc_score

In [8]:
#salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
# G_df = pd.DataFrame(index = future_connections.index)
# G_df['PAM'] = pd.Series([pa[-1] for pa in nx.preferential_attachment(G,G_df.index)], index = G_df.index)
# G_df['CommonNeigh'] = pd.Series( [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in G_df.index], index = G_df.index)
# G_df['JacCoeff'] = pd.Series([jc[-1] for jc in nx.jaccard_coefficient(G,G_df.index)], index = G_df.index)
# G_df['RAI'] = pd.Series([rai[-1] for rai in nx.resource_allocation_index(G,G_df.index)], index = G_df.index)
# G_df['FutureConnections']= future_connections[future_connections.columns[0]]
# G_df.head(20)

In [13]:
def new_connections_predictions():
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler,RobustScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve,auc
    ##############################################################################
    # DATA LOADING
    #############################################################################
  
    G_df = pd.DataFrame(index = future_connections.index)
   
    ## Features = ['PAM','CommonNeigh','JaccCoeff','RAI']
    G_df['PAM'] = pd.Series([pa[-1] for pa in nx.preferential_attachment(G,G_df.index)], index = G_df.index)
    G_df['CommonNeigh'] = pd.Series( [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in G_df.index], index = G_df.index)
    G_df['JaccCoeff'] = pd.Series([jc[-1] for jc in nx.jaccard_coefficient(G,G_df.index)], index = G_df.index)
    G_df['RAI'] = pd.Series([rai[-1] for rai in nx.resource_allocation_index(G,G_df.index)], index = G_df.index)
    G_df['FutureConnections']= future_connections[future_connections.columns[0]]

    ##############################################################################
    # PRE - PROCESSING
    ##############################################################################
    
    # Drop NaN values
    G_df_cleaned = G_df.dropna()
    
    ##############################################################################
    # MODEL BUILDING --> 'SVC' Model Chosen
    ##############################################################################
    
    # Step 1: Defining and scaling X and y #
    
    ## Training
    X = pd.DataFrame( G_df_cleaned[['PAM','CommonNeigh','JaccCoeff','RAI']])
    y = pd.DataFrame( G_df_cleaned['FutureConnections'])
    
    ## Validations
    X_val = pd.DataFrame(G_df[G_df.isnull().any(axis=1)][['PAM','CommonNeigh','JaccCoeff','RAI']])
    y_val = pd.DataFrame(G_df[G_df.isnull().any(axis=1)]['FutureConnections'])
    
    # Aditional Step: Normalizing and scaling
    scaler = RobustScaler()
    
    X_scaled = scaler.fit_transform(X)
    X_val_scaled = scaler.fit_transform(X_val)
    
    # Step 2: Train&Test Split #
    
    X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,random_state = 42)
    
    # Step 3: Model definition #
    # clf = LogisticRegression()
    clf = RandomForestClassifier(n_estimators = 50)
    
    # Step 4: Model fitting #
    clf.fit(X_scaled,y)
    
    # Step 5: Model prediction and evaluation #
    
    ## Test
    y_pred = clf.predict_proba(X_test)[:,-1]

    fpr,tpr, thresh = roc_curve(y_test,y_pred)
    auc_score = auc(fpr,tpr)
    
    # Prediction of nwe entries
    y_pred_val = pd.DataFrame( clf.predict_proba(X_val_scaled), index = X_val.index)
    
    return y_pred_val[1]

In [12]:
#new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


(107, 348)    0.000000
(542, 751)    0.006437
(20, 426)     0.600000
(50, 989)     0.010521
(942, 986)    0.008748
(324, 857)    0.019487
(13, 710)     0.000000
(19, 271)     0.120000
(319, 878)    0.014212
(659, 707)    0.040870
(49, 843)     0.007942
(208, 893)    0.007116
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000148
(292, 618)    0.040000
(239, 689)    0.009855
(359, 373)    0.000000
(53, 523)     0.000000
(276, 984)    0.011777
(202, 997)    0.003474
(604, 619)    0.000000
(270, 911)    0.012980
(261, 481)    0.040000
(200, 450)    0.640000
(213, 634)    0.011843
(644, 735)    0.080000
(346, 553)    0.026151
(521, 738)    0.007395
(422, 953)    0.000000
                ...   
(672, 848)    0.012980
(28, 127)     1.000000
(202, 661)    0.016984
(54, 195)     1.000000
(295, 864)    0.016337
(814, 936)    0.000000
(839, 874)    0.008748
(139, 843)    0.022206
(461, 544)    0.000000
(68, 487)     0.000000
(622, 932)    0.046127
(504, 936)    0.000000
(479, 528) 